In [1]:
import os
import pandas as pd
import numpy as np
import re
from catboost import CatBoostClassifier

In [2]:
from splitting import combine_files_into_dfs

In [3]:
FOLDER = '/home/alexey.osipov/playground/eco_geo/prototype/'

In [4]:
X_train, y_train = combine_files_into_dfs('train.txt', FOLDER)

In [5]:
X_train.drop(['Line', 'Intr_dist'], inplace = True, axis = 1)
y_train = y_train['Intr']

In [6]:
model_1 = CatBoostClassifier(iterations=10)

In [7]:
model_1.fit(X_train, y_train)

Learning rate set to 0.5
0:	learn: 0.2353313	total: 356ms	remaining: 3.2s
1:	learn: 0.1719446	total: 579ms	remaining: 2.32s
2:	learn: 0.1515386	total: 834ms	remaining: 1.95s
3:	learn: 0.1374955	total: 1.11s	remaining: 1.66s
4:	learn: 0.1304128	total: 1.32s	remaining: 1.32s
5:	learn: 0.1253928	total: 1.55s	remaining: 1.03s
6:	learn: 0.1183791	total: 1.77s	remaining: 761ms
7:	learn: 0.1145443	total: 2.03s	remaining: 509ms
8:	learn: 0.1114201	total: 2.28s	remaining: 254ms
9:	learn: 0.1072179	total: 2.52s	remaining: 0us


In [8]:
model_1.save_model('catboost_model_1_for_intr')

In [9]:
X_train_filtered = X_train[['X', 'Y', 'Gr_4_5', 'K', 'MED']]

In [10]:
model_2 = CatBoostClassifier(iterations=10)

In [11]:
model_2.fit(X_train_filtered, y_train)

Learning rate set to 0.5
0:	learn: 0.3924743	total: 150ms	remaining: 1.35s
1:	learn: 0.2801753	total: 285ms	remaining: 1.14s
2:	learn: 0.2220359	total: 420ms	remaining: 980ms
3:	learn: 0.1897426	total: 554ms	remaining: 831ms
4:	learn: 0.1697729	total: 694ms	remaining: 694ms
5:	learn: 0.1538214	total: 835ms	remaining: 557ms
6:	learn: 0.1465058	total: 995ms	remaining: 427ms
7:	learn: 0.1411196	total: 1.11s	remaining: 278ms
8:	learn: 0.1351086	total: 1.28s	remaining: 142ms
9:	learn: 0.1322330	total: 1.38s	remaining: 0us


In [12]:
model_2.save_model('catboost_model_2_for_intr')

In [13]:
def blend_file(filename = 'pred_data_8_56.csv',
               foldername = '/home/alexey.osipov/playground/eco_geo/prototype/data/',
               targetname = 'Intr',
               keycolumns = ['X', 'Y', 'Gr_4_5', 'K', 'MED'],
               model1 = model_1,
               model2 = model_2):
    raw_filename = re.sub('pred_', '', filename)
    tab_features = pd.read_csv(foldername + 'features/' + raw_filename)
    tab_features = tab_features.iloc[:,2:]
    tab_features_filtered = tab_features[keycolumns]
    tab_masks = pd.read_csv(foldername + 'masks/' + re.sub('data', 'masks', raw_filename))
    tab_results_deeplab = pd.read_csv(foldername + 'results/' + filename)
    tab_masks_deeplab = pd.read_csv(foldername + 'results/' + re.sub('pred_', 'target_', filename))
    assert np.all(tab_masks[targetname].values == tab_masks_deeplab['target'].values)
    res1 = model1.predict(tab_features)
    res2 = model2.predict(tab_features_filtered)
    tab_results_deeplab['model1'] = res1
    tab_results_deeplab['model2'] = res2
    tab_results_deeplab['model'] = round((tab_results_deeplab['prediction'] +\
        tab_results_deeplab['model1'] + tab_results_deeplab['model2'])/3)
    tab_results_deeplab['prediction'] = tab_masks_deeplab['target']
    numerator = len(tab_results_deeplab[(tab_results_deeplab['model'] == 1) & (tab_results_deeplab['prediction'] == 1)])
    denominator = len(tab_results_deeplab[(tab_results_deeplab['model'] == 1) | (tab_results_deeplab['prediction'] == 1)])
    return numerator, denominator

In [14]:
file_list = os.listdir('/home/alexey.osipov/playground/eco_geo/prototype/data/results')
file_list = [f for f in file_list if 'pred_' in f]

In [15]:
numerators = []
denominators = []
for f in file_list:
    print(f)
    numerator, denominator = blend_file(f)
    numerators.append(numerator)
    denominators.append(denominator)

pred_data_39_25.csv
pred_data_55_7.csv
pred_data_48_23.csv
pred_data_69_8.csv
pred_data_15_24.csv
pred_data_16_24.csv
pred_data_19_46.csv
pred_data_3_54.csv
pred_data_21_28.csv
pred_data_46_16.csv
pred_data_22_47.csv
pred_data_21_39.csv
pred_data_18_47.csv
pred_data_59_4.csv
pred_data_57_7.csv
pred_data_30_21.csv
pred_data_22_48.csv
pred_data_20_45.csv
pred_data_71_15.csv
pred_data_52_15.csv
pred_data_28_35.csv
pred_data_50_21.csv
pred_data_42_4.csv
pred_data_8_45.csv
pred_data_7_46.csv
pred_data_9_47.csv
pred_data_54_7.csv
pred_data_33_1.csv
pred_data_27_36.csv
pred_data_49_22.csv
pred_data_22_18.csv
pred_data_58_7.csv
pred_data_27_20.csv
pred_data_33_29.csv
pred_data_8_39.csv
pred_data_39_26.csv
pred_data_27_19.csv
pred_data_56_8.csv
pred_data_61_8.csv
pred_data_22_31.csv
pred_data_47_16.csv
pred_data_63_25.csv
pred_data_40_26.csv
pred_data_6_33.csv
pred_data_29_35.csv
pred_data_32_12.csv
pred_data_33_31.csv
pred_data_15_27.csv
pred_data_19_29.csv
pred_data_43_6.csv
pred_data_30_14.c

pred_data_31_23.csv
pred_data_46_15.csv
pred_data_30_39.csv
pred_data_31_6.csv
pred_data_29_8.csv
pred_data_21_48.csv
pred_data_10_36.csv
pred_data_23_16.csv
pred_data_19_30.csv
pred_data_59_23.csv
pred_data_48_21.csv
pred_data_20_47.csv
pred_data_28_8.csv
pred_data_41_3.csv
pred_data_52_17.csv
pred_data_20_27.csv
pred_data_70_6.csv
pred_data_14_25.csv
pred_data_33_19.csv


In [16]:
sum(numerators)/sum(denominators)

0.6349532383532908